In [14]:
import os
import cv2
import json
from datetime import datetime
from ultralytics import YOLO
from tqdm.notebook import tqdm
from dataclasses import dataclass, field, asdict
from typing import List, Optional, Dict, Set
from enum import Enum

class FilterMode(Enum):
    ALL = "all"
    INCLUDE = "include"
    EXCLUDE = "exclude"

@dataclass
class PipelineConfig:
    model_detect_path: str = 'data/model/yolo_nano_detect_v12.pt.pt'
    model_classify_path: str = 'data/model/yolo_small_classify_v15.pt.pt'
    video_path: str = ''
    output_base_dir: Optional[str] = None

    # Detection settings
    detection_confidence: float = 0.7
    detection_filter_mode: FilterMode = FilterMode.ALL
    detection_filter_classes: List[str] = field(default_factory=list)

    # Classification settings
    classification_confidence: float = 0.5
    classification_filter_mode: FilterMode = FilterMode.ALL
    classification_filter_classes: List[str] = field(default_factory=list)

    # Processing settings
    frame_skip: int = 100
    min_crop_size: int = 5
    save_format: str = "jpg"
    save_quality: int = 95

    # ⭐ NEW: Frame saving options
    save_full_frames: bool = True           # Save the complete frame
    save_crops: bool = True                  # Save cropped classifications
    draw_boxes_on_frame: bool = False         # Draw bounding boxes on saved frames
    box_thickness: int = 2                   # Bounding box line thickness

    # Metadata
    save_metadata: bool = True

    # Debug settings
    debug_mode: bool = False
    debug_max_frames: int = 500

    def __post_init__(self):
        if self.output_base_dir is None and self.video_path:
            video_dir = os.path.dirname(self.video_path)
            self.output_base_dir = os.path.join(video_dir, 'output')


class DetectClassifyPipeline:
    """
    Output Structure:
    output/
    └── Detection/
        └── [detect_class_name]/
            ├── Pure/                    # Full frames with this detection class
            │   ├── 0001.jpg
            │   └── 0002.jpg
            └── Classes/
                └── [classify_class_name]/  # Cropped & classified objects
                    ├── 0001.jpg
                    └── 0002.jpg
    """

    # Colors for bounding boxes (BGR format)
    COLORS = [
        (0, 255, 0),    # Green
        (255, 0, 0),    # Blue
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 0, 255),  # Purple
        (255, 128, 0),  # Orange
    ]

    def __init__(self, config: PipelineConfig):
        self.config = config

        # Counters for sequential naming
        # Structure: {detect_class: {classify_class: count}} for crops
        # Structure: {detect_class: count} for pure frames
        self.crop_counters: Dict[str, Dict[str, int]] = {}
        self.frame_counters: Dict[str, int] = {}

        # Track which frames we've already saved per detection class
        # to avoid saving the same frame multiple times
        self.saved_frames: Dict[str, Set[int]] = {}

        self.metadata_log: List[Dict] = []

        # Debug stats
        self.debug_stats = {
            'frames_processed': 0,
            'total_detections': 0,
            'filtered_by_detection_class': 0,
            'filtered_by_crop_size': 0,
            'filtered_by_classification_confidence': 0,
            'filtered_by_classification_class': 0,
            'frames_saved': 0,
            'crops_saved': 0,
            'save_failures': 0,
        }

        # Load models
        print("🔧 Loading YOLO models...")
        self.model_detect = YOLO(config.model_detect_path)
        self.model_classify = YOLO(config.model_classify_path, task='classify')

        self.detect_names = self.model_detect.names
        self.classify_names = self.model_classify.names

        # Get color mapping for detection classes
        self.class_colors = {
            name: self.COLORS[idx % len(self.COLORS)]
            for idx, name in self.detect_names.items()
        }

        self._print_config()
        self._setup_directories()

    def _print_config(self):
        print("\n" + "="*60)
        print("📋 PIPELINE CONFIGURATION")
        print("="*60)

        print("\n📦 Detection classes available:")
        for idx, name in self.detect_names.items():
            print(f"   {idx}: '{name}'")

        print("\n🏷️  Classification classes available:")
        for idx, name in self.classify_names.items():
            print(f"   {idx}: '{name}'")

        print(f"\n⚙️  Filter Settings:")
        print(f"   Detection mode: {self.config.detection_filter_mode.value}")
        print(f"   Detection classes: {self.config.detection_filter_classes or 'ALL'}")
        print(f"   Detection confidence: {self.config.detection_confidence}")
        print(f"   Classification mode: {self.config.classification_filter_mode.value}")
        print(f"   Classification classes: {self.config.classification_filter_classes or 'ALL'}")
        print(f"   Classification confidence: {self.config.classification_confidence}")

        print(f"\n💾 Save Settings:")
        print(f"   Save full frames: {self.config.save_full_frames}")
        print(f"   Save crops: {self.config.save_crops}")
        print(f"   Draw boxes on frames: {self.config.draw_boxes_on_frame}")

        print(f"\n📁 Output Structure:")
        print(f"   {self.config.output_base_dir}/")
        print(f"   └── Detection/")
        print(f"       └── [detect_class]/")
        print(f"           ├── Pure/          # Full frames")
        print(f"           └── Classes/")
        print(f"               └── [classify_class]/  # Crops")

    def _setup_directories(self):
        """Create output directory structure."""
        os.makedirs(self.config.output_base_dir, exist_ok=True)

        # Test write access
        test_file = os.path.join(self.config.output_base_dir, '_test.tmp')
        try:
            with open(test_file, 'w') as f:
                f.write('test')
            os.remove(test_file)
            print(f"\n✅ Output directory ready: {self.config.output_base_dir}")
        except Exception as e:
            print(f"\n❌ OUTPUT DIRECTORY NOT WRITABLE: {e}")

    def _get_pure_frame_path(self, detect_class: str) -> str:
        """Get path for saving full frame under detection class."""
        # Structure: output/Detection/[detect_class]/Pure/0001.jpg
        pure_dir = os.path.join(
            self.config.output_base_dir,
            "Detection",
            detect_class,
            "Pure"
        )
        os.makedirs(pure_dir, exist_ok=True)

        # Initialize counter if needed
        if detect_class not in self.frame_counters:
            self.frame_counters[detect_class] = 1

        base = os.path.splitext(os.path.basename(self.config.video_path))[0]
        filename = f"{base}_{self.frame_counters[detect_class]:04d}.{self.config.save_format}"
        self.frame_counters[detect_class] += 1

        return os.path.join(pure_dir, filename)

    def _get_crop_path(self, detect_class: str, classify_class: str) -> str:
        """Get path for saving cropped classification."""
        # Structure: output/Detection/[detect_class]/Classes/[classify_class]/0001.jpg
        crop_dir = os.path.join(
            self.config.output_base_dir,
            "Detection",
            detect_class,
            "Classes",
            classify_class
        )
        os.makedirs(crop_dir, exist_ok=True)

        # Initialize counters if needed
        if detect_class not in self.crop_counters:
            self.crop_counters[detect_class] = {}
        if classify_class not in self.crop_counters[detect_class]:
            self.crop_counters[detect_class][classify_class] = 1

        filename = f"{self.crop_counters[detect_class][classify_class]:04d}.{self.config.save_format}"
        self.crop_counters[detect_class][classify_class] += 1

        return os.path.join(crop_dir, filename)

    def _should_process_detection(self, class_name: str) -> bool:
        mode = self.config.detection_filter_mode
        filter_classes = self.config.detection_filter_classes

        if mode == FilterMode.ALL:
            return True
        elif mode == FilterMode.INCLUDE:
            return class_name in filter_classes
        elif mode == FilterMode.EXCLUDE:
            return class_name not in filter_classes
        return True

    def _should_save_classification(self, class_name: str, confidence: float) -> tuple:
        """Returns (should_save, reason_if_not)"""
        if confidence < self.config.classification_confidence:
            return False, f"confidence {confidence:.3f} < {self.config.classification_confidence}"

        mode = self.config.classification_filter_mode
        filter_classes = self.config.classification_filter_classes

        if mode == FilterMode.ALL:
            return True, ""
        elif mode == FilterMode.INCLUDE:
            if class_name in filter_classes:
                return True, ""
            return False, f"'{class_name}' not in {filter_classes}"
        elif mode == FilterMode.EXCLUDE:
            if class_name not in filter_classes:
                return True, ""
            return False, f"'{class_name}' is excluded"
        return True, ""

    def _save_image(self, path: str, image) -> bool:
        """Save image with error handling."""
        try:
            if self.config.save_format == "jpg":
                success = cv2.imwrite(path, image,
                    [cv2.IMWRITE_JPEG_QUALITY, self.config.save_quality])
            else:
                success = cv2.imwrite(path, image)

            if not success:
                print(f"❌ cv2.imwrite failed: {path}")
                self.debug_stats['save_failures'] += 1
                return False
            return True

        except Exception as e:
            print(f"❌ Save error: {e}")
            self.debug_stats['save_failures'] += 1
            return False

    def _draw_boxes_on_frame(self, frame, detections_info: List[Dict]):
        """Draw bounding boxes and labels on frame."""
        frame_copy = frame.copy()

        for det in detections_info:
            x1, y1, x2, y2 = det['bbox']
            detect_class = det['detect_class']
            classify_class = det['classify_class']
            classify_conf = det['classify_conf']

            color = self.class_colors.get(detect_class, (0, 255, 0))

            # Draw rectangle
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), color, self.config.box_thickness)

            # Create label
            label = f"{classify_class} ({classify_conf:.2f})"

            # Get label size for background
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            thickness = 1
            (label_w, label_h), baseline = cv2.getTextSize(label, font, font_scale, thickness)

            # Draw label background
            cv2.rectangle(frame_copy,
                         (x1, y1 - label_h - 10),
                         (x1 + label_w + 5, y1),
                         color, -1)

            # Draw label text
            cv2.putText(frame_copy, label,
                       (x1 + 2, y1 - 5),
                       font, font_scale, (255, 255, 255), thickness)

        return frame_copy

    def process_frame(self, frame, frame_number: int) -> Dict[str, int]:
        """
        Process a single frame.
        Returns dict with counts: {'frames_saved': n, 'crops_saved': m}
        """
        if frame is None:
            return {'frames_saved': 0, 'crops_saved': 0}

        self.debug_stats['frames_processed'] += 1
        results = {'frames_saved': 0, 'crops_saved': 0}

        # Run detection
        results_detect = self.model_detect(
            frame,
            conf=self.config.detection_confidence,
            verbose=False
        )

        if not results_detect or not results_detect[0].boxes:
            return results

        detections = results_detect[0].boxes
        self.debug_stats['total_detections'] += len(detections)

        # Group detections by detection class
        # Structure: {detect_class: [list of detection info]}
        detections_by_class: Dict[str, List[Dict]] = {}

        if self.config.debug_mode:
            print(f"\n🎬 Frame {frame_number}: {len(detections)} detections")

        for idx, box in enumerate(detections):
            detect_class_id = int(box.cls[0])
            detect_class_name = self.detect_names[detect_class_id]
            detect_confidence = float(box.conf[0])

            if self.config.debug_mode:
                print(f"   [{idx}] Detected: '{detect_class_name}' (conf: {detect_confidence:.3f})")

            # Filter by detection class
            if not self._should_process_detection(detect_class_name):
                self.debug_stats['filtered_by_detection_class'] += 1
                if self.config.debug_mode:
                    print(f"      ⏭️  Skipped: detection class filter")
                continue

            # Get bounding box and crop
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)

            crop_img = frame[y1:y2, x1:x2]

            # Validate crop size
            if (crop_img.size == 0 or
                crop_img.shape[0] < self.config.min_crop_size or
                crop_img.shape[1] < self.config.min_crop_size):
                self.debug_stats['filtered_by_crop_size'] += 1
                if self.config.debug_mode:
                    print(f"      ⏭️  Skipped: crop too small")
                continue

            # Run classification
            results_classify = self.model_classify(crop_img, verbose=False)

            classify_class_id = results_classify[0].probs.top1
            classify_class_name = self.classify_names[classify_class_id]
            classify_confidence = float(results_classify[0].probs.top1conf)

            if self.config.debug_mode:
                print(f"      🏷️  Classified: '{classify_class_name}' (conf: {classify_confidence:.3f})")

            # Filter by classification
            should_save, reason = self._should_save_classification(classify_class_name, classify_confidence)
            if not should_save:
                if classify_confidence < self.config.classification_confidence:
                    self.debug_stats['filtered_by_classification_confidence'] += 1
                else:
                    self.debug_stats['filtered_by_classification_class'] += 1
                if self.config.debug_mode:
                    print(f"      ⏭️  Skipped: {reason}")
                continue

            # Store detection info
            det_info = {
                'bbox': [x1, y1, x2, y2],
                'detect_class': detect_class_name,
                'detect_conf': detect_confidence,
                'classify_class': classify_class_name,
                'classify_conf': classify_confidence,
                'crop_img': crop_img,
            }

            if detect_class_name not in detections_by_class:
                detections_by_class[detect_class_name] = []
            detections_by_class[detect_class_name].append(det_info)

        # Now save frames and crops organized by detection class
        for detect_class, det_list in detections_by_class.items():

            # Initialize saved frames tracking
            if detect_class not in self.saved_frames:
                self.saved_frames[detect_class] = set()

            # Save full frame (once per detection class per frame)
            if self.config.save_full_frames:
                if frame_number not in self.saved_frames[detect_class]:

                    if self.config.draw_boxes_on_frame:
                        frame_to_save = self._draw_boxes_on_frame(frame, det_list)
                    else:
                        frame_to_save = frame

                    frame_path = self._get_pure_frame_path(detect_class)

                    if self._save_image(frame_path, frame_to_save):
                        results['frames_saved'] += 1
                        self.debug_stats['frames_saved'] += 1
                        self.saved_frames[detect_class].add(frame_number)

                        if self.config.debug_mode:
                            print(f"      💾 Saved frame: {frame_path}")

            # Save crops
            if self.config.save_crops:
                for det_info in det_list:
                    crop_path = self._get_crop_path(
                        detect_class,
                        det_info['classify_class']
                    )

                    if self._save_image(crop_path, det_info['crop_img']):
                        results['crops_saved'] += 1
                        self.debug_stats['crops_saved'] += 1

                        if self.config.debug_mode:
                            print(f"      💾 Saved crop: {crop_path}")

                    # Log metadata
                    if self.config.save_metadata:
                        self.metadata_log.append({
                            'frame': frame_number,
                            'crop_file': crop_path,
                            'detection_class': det_info['detect_class'],
                            'detection_confidence': round(det_info['detect_conf'], 4),
                            'classification_class': det_info['classify_class'],
                            'classification_confidence': round(det_info['classify_conf'], 4),
                            'bbox': det_info['bbox']
                        })

        return results

    def process_video(self):
        """Process the entire video."""
        if not os.path.exists(self.config.video_path):
            print(f"❌ ERROR: Video not found: {self.config.video_path}")
            return

        cap = cv2.VideoCapture(self.config.video_path)
        if not cap.isOpened():
            print(f"❌ ERROR: Could not open video: {self.config.video_path}")
            return

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        print(f"\n🎥 Video: {os.path.basename(self.config.video_path)}")
        print(f"   Frames: {frame_count} | FPS: {fps:.2f}")
        print(f"   Processing every {self.config.frame_skip} frames")
        print(f"   Estimated frames to process: {frame_count // self.config.frame_skip}")

        if self.config.debug_mode:
            print(f"   🐛 DEBUG MODE: Stopping after {self.config.debug_max_frames} frames")

        total_frames_saved = 0
        total_crops_saved = 0
        frame_number = 0

        max_frames = self.config.debug_max_frames if self.config.debug_mode else frame_count

        with tqdm(total=min(frame_count, max_frames), desc="Processing") as pbar:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_number += 1
                pbar.update(1)

                if self.config.debug_mode and frame_number > self.config.debug_max_frames:
                    print(f"\n🛑 Debug mode: Stopping after {self.config.debug_max_frames} frames")
                    break

                if frame_number % self.config.frame_skip != 0:
                    continue

                result = self.process_frame(frame, frame_number)
                total_frames_saved += result['frames_saved']
                total_crops_saved += result['crops_saved']

                pbar.set_postfix({
                    'frames': total_frames_saved,
                    'crops': total_crops_saved
                })

        cap.release()

        # Save metadata
        if self.config.save_metadata and self.metadata_log:
            metadata_path = os.path.join(self.config.output_base_dir, 'metadata.json')
            with open(metadata_path, 'w') as f:
                json.dump({
                    'config': asdict(self.config),
                    'crops': self.metadata_log,
                    'frame_counts': self.frame_counters,
                    'crop_counts': self.crop_counters,
                    'debug_stats': self.debug_stats,
                    'timestamp': datetime.now().isoformat()
                }, f, indent=2, default=str)
            print(f"\n📄 Metadata saved: {metadata_path}")

        self._print_summary()

    def _print_summary(self):
        print("\n" + "="*60)
        print("📊 PROCESSING SUMMARY")
        print("="*60)

        stats = self.debug_stats

        print(f"\n📈 Statistics:")
        print(f"   Frames processed:              {stats['frames_processed']}")
        print(f"   Total detections:              {stats['total_detections']}")
        print(f"   Filtered (detection class):    {stats['filtered_by_detection_class']}")
        print(f"   Filtered (crop size):          {stats['filtered_by_crop_size']}")
        print(f"   Filtered (classify conf):      {stats['filtered_by_classification_confidence']}")
        print(f"   Filtered (classify class):     {stats['filtered_by_classification_class']}")

        print(f"\n💾 Saved:")
        print(f"   Full frames saved:             {stats['frames_saved']} ✅")
        print(f"   Crops saved:                   {stats['crops_saved']} ✅")
        print(f"   Save failures:                 {stats['save_failures']} ❌")

        # Print breakdown by class
        if self.frame_counters:
            print(f"\n📁 Frames per detection class:")
            for detect_class, count in self.frame_counters.items():
                print(f"   {detect_class}: {count - 1} frames")

        if self.crop_counters:
            print(f"\n📁 Crops breakdown:")
            for detect_class, classify_counts in self.crop_counters.items():
                print(f"   {detect_class}/")
                for classify_class, count in classify_counts.items():
                    print(f"      └── {classify_class}: {count - 1} crops")

        print(f"\n📂 Output folder: {self.config.output_base_dir}")

        # Show directory structure
        print("\n📂 Directory structure created:")
        self._print_directory_tree()

    def _print_directory_tree(self, max_files: int = 3):
        """Print the output directory structure."""
        for root, dirs, files in os.walk(self.config.output_base_dir):
            level = root.replace(self.config.output_base_dir, '').count(os.sep)
            indent = '   ' * level
            folder_name = os.path.basename(root)
            file_count = len(files)

            if file_count > 0:
                print(f"{indent}📁 {folder_name}/ ({file_count} files)")
            else:
                print(f"{indent}📁 {folder_name}/")


# ==============================================================================
# EXAMPLE CONFIGURATIONS
# ==============================================================================

# Configuration 1: Save everything (both frame types, all classes)
config_all = PipelineConfig(
    video_path='D:\\Recordings\\New_Recordings\\GREEN.mp4',
    detection_filter_mode=FilterMode.ALL,
    classification_filter_mode=FilterMode.ALL,
    detection_confidence=0.7,
    classification_confidence=0.7,
    save_full_frames=True,
    save_crops=True,
    draw_boxes_on_frame=True,
    frame_skip=60,
)

# Configuration 3: Both bag types, specific colors only
config_specific_colors = PipelineConfig(
    video_path='D:\\Recordings\\New_Recordings\\GREEN.mp4',
    detection_filter_mode=FilterMode.INCLUDE,
    detection_filter_classes=['bread-bag-opened', 'bread-bag-closed'],
    classification_filter_mode=FilterMode.INCLUDE,
    classification_filter_classes=['Red_Yellow', 'Blue_Yellow', 'Green_Yellow'],
    detection_confidence=0.7,
    classification_confidence=0.8,
    save_full_frames=True,
    save_crops=True,
    draw_boxes_on_frame=True,
    frame_skip=60,
)

# Configuration 4: Only crops, no frames (faster, less disk space)
config_crops_only = PipelineConfig(
    video_path='D:\\Recordings\\New_Recordings\\GREEN.mp4',
    detection_filter_mode=FilterMode.ALL,
    classification_filter_mode=FilterMode.ALL,
    detection_confidence=0.7,
    classification_confidence=0.8,
    save_full_frames=False,  # Don't save full frames
    save_crops=True,
    frame_skip=60,
)


# ==============================================================================
# RUN PIPELINE
# ==============================================================================

# Choose your configuration
config = PipelineConfig(
    video_path='D:\\Recordings\\New_Recordings\\GREEN.mp4',

    # Detection settings
    detection_filter_mode=FilterMode.INCLUDE,
    detection_filter_classes=['bread-bag-closed'],  # Both types
    detection_confidence=0.7,

    # Classification settings
    classification_filter_mode=FilterMode.ALL,  # All color types
    classification_confidence=0.8,

    # Save settings
    save_full_frames=True,
    save_crops=True,
    draw_boxes_on_frame=True,

    # Processing
    frame_skip=10,

    # Debug (set to False for full run)
    debug_mode=True,
    debug_max_frames=20000,
)

# Configuration 2: Only opened bags

config_opened = PipelineConfig(
    video_path='D:\\Recordings\\2026_02_05\\2026_02_11\\mp4\\output_2026-02-11_02-36-34.mp4',
    detection_filter_mode=FilterMode.INCLUDE,
    detection_filter_classes=['bread-bag'],
    detection_confidence=0.75,
    classification_confidence=0.75,
    save_full_frames=True,
    save_crops=True,
    draw_boxes_on_frame=False,
    frame_skip=14,
    debug_mode=False,
    debug_max_frames=10000
)

pipeline = DetectClassifyPipeline(config_opened)
pipeline.process_video()
# Notebook helper: process all .mp4 files in a directory with your existing pipeline.
# Paste this into a notebook cell (assumes config_opened, PipelineConfig, DetectClassifyPipeline are already defined
# in the notebook or are importable).

🔧 Loading YOLO models...


Exception ignored in: <function tqdm.__del__ at 0x000001D67AFA80E0>
Traceback (most recent call last):
  File "C:\0001_MyFiles\0016_Projects\0002_ProjectBased\0012_ConvuyerBreadBagCounterSystem\.venv\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\0001_MyFiles\0016_Projects\0002_ProjectBased\0012_ConvuyerBreadBagCounterSystem\.venv\Lib\site-packages\tqdm\notebook.py", line 282, in close
    self.disp(bar_style='success', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'



📋 PIPELINE CONFIGURATION

📦 Detection classes available:
   0: 'bread-bag'

🏷️  Classification classes available:
   0: 'Blue_Yellow'
   1: 'Bran'
   2: 'Brown_Orange_Family'
   3: 'Green_Yellow'
   4: 'Red_Yellow'
   5: 'Rejected'
   6: 'Wheatberry'

⚙️  Filter Settings:
   Detection mode: include
   Detection classes: ['bread-bag']
   Detection confidence: 0.75
   Classification mode: all
   Classification classes: ALL
   Classification confidence: 0.75

💾 Save Settings:
   Save full frames: True
   Save crops: True
   Draw boxes on frames: False

📁 Output Structure:
   D:\Recordings\2026_02_05\2026_02_11\mp4\output/
   └── Detection/
       └── [detect_class]/
           ├── Pure/          # Full frames
           └── Classes/
               └── [classify_class]/  # Crops

✅ Output directory ready: D:\Recordings\2026_02_05\2026_02_11\mp4\output

🎥 Video: output_2026-02-11_02-36-34.mp4
   Frames: 25200 | FPS: 14.00
   Processing every 14 frames
   Estimated frames to process: 1800


Processing:   0%|          | 0/25200 [00:00<?, ?it/s]


📄 Metadata saved: D:\Recordings\2026_02_05\2026_02_11\mp4\output\metadata.json

📊 PROCESSING SUMMARY

📈 Statistics:
   Frames processed:              1800
   Total detections:              2081
   Filtered (detection class):    0
   Filtered (crop size):          0
   Filtered (classify conf):      356
   Filtered (classify class):     0

💾 Saved:
   Full frames saved:             1258 ✅
   Crops saved:                   1725 ✅
   Save failures:                 0 ❌

📁 Frames per detection class:
   bread-bag: 1258 frames

📁 Crops breakdown:
   bread-bag/
      └── Brown_Orange_Family: 1341 crops
      └── Rejected: 99 crops
      └── Blue_Yellow: 102 crops
      └── Bran: 21 crops
      └── Red_Yellow: 1 crops
      └── Wheatberry: 18 crops
      └── Green_Yellow: 143 crops

📂 Output folder: D:\Recordings\2026_02_05\2026_02_11\mp4\output

📂 Directory structure created:
📁 output/ (1 files)
   📁 Detection/
      📁 bread-bag/
         📁 Classes/
            📁 Blue_Yellow/ (102 files)
   

In [ ]:
import copy
import traceback
from pathlib import Path

# Optional: nice progress bar if tqdm is installed; falls back to a simple iterator.
try:
    from tqdm.notebook import tqdm
except Exception:
    try:
        from tqdm import tqdm  # terminal tqdm
    except Exception:
        tqdm = lambda x, **kw: x  # fallback: identity

def process_all_mp4s(
    mp4_dir="D:\\Recordings\\2026_02_05\\2026_02_09\\mp4",
    recursive=False,
    dry_run=False,
    stop_on_error=False,
    show_progress=True,
):
    """
    Process every .mp4 in `mp4_dir` using your existing `config_opened` and
    `DetectClassifyPipeline`.

    Parameters
    - mp4_dir: directory to search for .mp4 files (default "mp4")
    - recursive: whether to search recursively (default False)
    - dry_run: if True, only lists files found (no processing)
    - stop_on_error: if True, stops on first exception
    - show_progress: use tqdm progress bar when available
    """
    # Ensure the base config/pipeline exist in notebook namespace or import them
    if "config_opened" not in globals():
        raise RuntimeError("config_opened not found in notebook globals. Define it or import it first.")
    if "DetectClassifyPipeline" not in globals():
        raise RuntimeError("DetectClassifyPipeline not found in notebook globals. Define it or import it first.")

    mp4_dir = Path(mp4_dir)
    if not mp4_dir.exists() or not mp4_dir.is_dir():
        raise FileNotFoundError(f"MP4 directory not found: {mp4_dir}")

    files = sorted(mp4_dir.rglob("*.mp4") if recursive else mp4_dir.glob("*.mp4"))
    files = list(files)
    if not files:
        print(f"No .mp4 files found in {mp4_dir} (recursive={recursive}).")
        return

    success = 0
    failed = 0

    iterator = tqdm(files) if show_progress else files
    for mp4_path in iterator:
        print(f"\n--- Processing: {mp4_path} ---")
        if dry_run:
            continue
        try:
            cfg = copy.deepcopy(config_opened)
            cfg.video_path = str(mp4_path)  # update only video path
            pipeline = DetectClassifyPipeline(cfg)
            pipeline.process_video()
            success += 1
        except Exception:
            failed += 1
            print(f"Error processing {mp4_path}:")
            traceback.print_exc()
            if stop_on_error:
                break

    print(f"\nFinished. Processed: {success + failed}, Success: {success}, Failed: {failed}")

# Example usage in notebook cells:
process_all_mp4s()                  # process files in ./mp4 (non-recursive)
# process_all_mp4s(mp4_dir="mp4", recursive=True)  # recursive
# process_all_mp4s(dry_run=True)      # list files only